<a href="https://colab.research.google.com/github/lala-lagrange/Julia-sublime/blob/master/bits_lung_cancer_eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#DOWNLOADING DATASET FROM KAGGLE

!pip install -q kaggle

from google.colab import files
files.upload()

! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

! chmod 600 ~/.kaggle/kaggle.json

#downloading the dataset and extracting files from it
!kaggle datasets download -d mohamedhanyyy/chest-ctscan-images
!unzip chest-ctscan-images.zip

In [16]:
        #IMPORTING PACKAGES
import numpy as np
import pandas as pd
import os
from pathlib import Path
import cv2

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, classification_report, roc_curve
from sklearn.model_selection import learning_curve, cross_val_score, GridSearchCV
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import RobustScaler,StandardScaler,MinMaxScaler

import warnings
warnings.filterwarnings('ignore')

## **EXTRACTING INFO ABOUT DATASET & CREATING A CSV FILE OUT OF IT**

In [5]:
                # Set directories
test_dir = "/content/Data/test"
train_dir = "/content/Data/train"
valid_dir = "/content/Data/valid"

                    #TRAIN-VAILD DATA AS CSV
train_val_images = []#creating an empty list
                #create csv for data
for folder in os.listdir(train_dir):#we got 4 folders in train directory...
    for file in os.listdir(os.path.join(train_dir,folder)):#for each file in a folder...
        image_arr = cv2.imread(os.path.join(train_dir,folder,file))#using opencv to read the image files...
        train_val_images.append({'image':file,
                       'type':folder,
                       'image_dimensions':image_arr.shape,
                       'image_height':image_arr.shape[0],
                       'image_width':image_arr.shape[1],
                       'set':'train'})#appending the list with properties of images, that we would like to do EDA

for folder in os.listdir(valid_dir):
    for file in os.listdir(os.path.join(valid_dir,folder)):
        image_arr = cv2.imread(os.path.join(valid_dir,folder,file))
        train_val_images.append({'image':file,
                       'type':folder,
                       'image_dimensions':image_arr.shape,
                       'image_height':image_arr.shape[0],
                       'image_width':image_arr.shape[1],
                       'set':'valid'})

#create dataframe of all data
train_val_df = pd.DataFrame(train_val_images)#converting the train_val list to a pandas dataframe
del train_val_images#removing the list, to save memory on the machine
#rename types (for easier readability)


train_val_df['type_extended'] = train_val_df['type']#creating a copy of 'type' column
#This is a lambda function (an anonymous function) that takes a single input 'string'.
# for e.g., It checks if the substring 'normal' exists in the input 'string'. If it does, it returns 'normal'; otherwise, it returns the original 'string'.
train_val_df['type'] = train_val_df['type'].apply(lambda string: 'squamous_cell_carcinoma' if 'squamous' in string else string)
train_val_df['type'] = train_val_df['type'].apply(lambda string: 'large_cell_carcinoma' if 'large' in string else string)
train_val_df['type'] = train_val_df['type'].apply(lambda string: 'adeno_carcinoma' if 'adeno' in string else string)
train_val_df['type'] = train_val_df['type'].apply(lambda string: 'normal' if 'normal' in string else string)
train_val_df.head(5)

train_val_df.to_csv('train-valid-image-data.csv',index=False)
print('[INFO] train-valid image data saved as csv file, for EDA')
# this dataset is a collection of data about images in train and valid directories
# this dataset will be futher used to perform EDA upon

                        #TEST DATA AS CSV
# uncomment the section below if u want test data EDA
test_images = []#creating an array for test directory
for folder in os.listdir(test_dir):
    for file in os.listdir(os.path.join(test_dir,folder)):
        image_arr = cv2.imread(os.path.join(test_dir,folder,file))
        test_images.append({'image':file,
                       'type':folder,
                       'image_dimensions':image_arr.shape,
                       'image_height':image_arr.shape[0],
                       'image_width':image_arr.shape[1],
                       'set':'test'})

test_df = pd.DataFrame(test_images)#converting the test list to a pandas dataframe
del test_images#removing the list, to save memory on the machine
#rename types (for easier readability)


test_df['type_extended'] = test_df['type']#creating a copy of 'type' column
#This is a lambda function (an anonymous function) that takes a single input 'string'.
# for e.g., It checks if the substring 'normal' exists in the input 'string'. If it does, it returns 'normal'; otherwise, it returns the original 'string'.
test_df['type'] = test_df['type'].apply(lambda string: 'squamous_cell_carcinoma' if 'squamous' in string else string)
test_df['type'] = test_df['type'].apply(lambda string: 'large_cell_carcinoma' if 'large' in string else string)
test_df['type'] = test_df['type'].apply(lambda string: 'adeno_carcinoma' if 'adeno' in string else string)
test_df['type'] = test_df['type'].apply(lambda string: 'normal' if 'normal' in string else string)
test_df.head(5)

test_df.to_csv('test-image-data.csv',index=False)
print('[INFO] test image data saved as csv file, for EDA')
#this dataset is a collection of data about images in train and valid directories
#this dataset will be futher used to perform EDA upon

[INFO] train-valid image data saved as csv file, for EDA
[INFO] test image data saved as csv file, for EDA


In [6]:
train_valid_data = pd.read_csv('/content/train-valid-image-data.csv')#LOADING THE CSV FILE
train_valid_data

,image,type,image_dimensions,image_height,image_width,set,type_extended
0,12 - Copy (3).png,normal,"(481, 619, 3)",481,619,train,normal
1,20.png,normal,"(270, 393, 3)",270,393,train,normal
2,19.png,normal,"(419, 538, 3)",419,538,train,normal
3,6.png,normal,"(552, 800, 3)",552,800,train,normal
4,7.png,normal,"(627, 940, 3)",627,940,train,normal
...,...,...,...,...,...,...,...
680,000111.png,squamous_cell_carcinoma,"(248, 350, 3)",248,350,valid,squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa
681,000117 (4).png,squamous_cell_carcinoma,"(271, 440, 3)",271,440,valid,squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa
682,000118 (5).png,squamous_cell_carcinoma,"(231, 410, 3)",231,410,valid,squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa
683,000117 (6).png,squamous_cell_carcinoma,"(245, 363, 3)",245,363,valid,squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa


In [7]:
print(train_valid_data.columns[train_valid_data.isna().any()])#checking if there's any NaN values in the dataset

Index([], dtype='object')


In [8]:
print(train_valid_data.info())#checking for basic information

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 685 entries, 0 to 684
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   image             685 non-null    object
 1   type              685 non-null    object
 2   image_dimensions  685 non-null    object
 3   image_height      685 non-null    int64 
 4   image_width       685 non-null    int64 
 5   set               685 non-null    object
 6   type_extended     685 non-null    object
dtypes: int64(2), object(5)
memory usage: 37.6+ KB
None


In [9]:
print(train_valid_data.describe())#checking for statistical information

       image_height  image_width
count    685.000000   685.000000
mean     302.912409   436.039416
std      107.160048   134.039784
min      110.000000   168.000000
25%      242.000000   376.000000
50%      271.000000   407.000000
75%      314.000000   438.000000
max      874.000000  1200.000000


In [10]:
print(train_valid_data['type'].value_counts())#distribution of 'type' column

adeno_carcinoma            218
squamous_cell_carcinoma    170
normal                     161
large_cell_carcinoma       136
Name: type, dtype: int64


In [11]:
print(train_valid_data['set'].value_counts())#distribution of 'set' column

train    613
valid     72
Name: set, dtype: int64


## **CATEGORICAL DISTRIBUTIONS(SET)**

In [12]:
import numpy as np
from google.colab import autoviz

def categorical_histogram(df, colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (8 * figscale, 4.8 * figscale)
  _, ax = plt.subplots(figsize=figsize)
  bars = df[colname].value_counts()
  plt.barh(bars.index, bars.values, color=sns.palettes.mpl_palette(mpl_palette_name))
  plt.title(colname)
  ax.spines[['top', 'right',]].set_visible(False)
  return autoviz.MplChart.from_current_mpl_state()

chart = categorical_histogram(train_valid_data, *['set'], **{})
chart

## **CATEGORICAL DISTRIBUTIONS(TYPE)**

In [13]:
import numpy as np
from google.colab import autoviz

def categorical_histogram(df, colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (8 * figscale, 4.8 * figscale)
  _, ax = plt.subplots(figsize=figsize)
  bars = df[colname].value_counts()
  plt.barh(bars.index, bars.values, color=sns.palettes.mpl_palette(mpl_palette_name))
  plt.title(colname)
  ax.spines[['top', 'right',]].set_visible(False)
  return autoviz.MplChart.from_current_mpl_state()

chart = categorical_histogram(train_valid_data, *['type'], **{})
chart

## **IMAGE HEIGHT DISTRIBUTION**

In [14]:
import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  figsize = (8 * figscale, 4 * figscale)
  _, ax = plt.subplots(figsize=figsize)
  plt.hist(df[colname], bins=num_bins, histtype='stepfilled')
  plt.ylabel('count')
  plt.title(colname)
  ax.spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(train_valid_data, *['image_height'], **{})
chart

## **IMAGE WIDTH DISTRIBUTION**

In [15]:
import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  figsize = (8 * figscale, 4 * figscale)
  _, ax = plt.subplots(figsize=figsize)
  plt.hist(df[colname], bins=num_bins, histtype='stepfilled')
  plt.ylabel('count')
  plt.title(colname)
  ax.spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(train_valid_data, *['image_width'], **{})
chart